In [10]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [11]:
# Access Camera
cap = cv2.VideoCapture(0)

# Detecting, Initializing and configuring the hands

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

In [12]:
# Accessing Speaker using Pycaw

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface,POINTER(IAudioEndpointVolume))

In [13]:
# Finding volume range

vol_min, vol_max = volume.GetVolumeRange()[:2]

In [19]:
# Capture an Image from the Camera

while True:
    success, img = cap.read()
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_RGB)

    # Checking for multiple hands
    lm_list = []
    if results.multi_hand_landmarks:
        for hand_land_mark in results.multi_hand_landmarks:
            for id,lm in enumerate(hand_land_mark.landmark):
                h,w,_ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append([id,cx,cy])
            mp_draw.draw_landmarks(img, hand_land_mark, mp_hands.HAND_CONNECTIONS)
    
    if lm_list != []:
        x1, y1 = lm_list[4][1], lm_list[4][2]
        x2, y2 = lm_list[8][1], lm_list[8][2]

        # Drawing a circle at the tip of the index finger and thumb
        cv2.circle(img, (x1,y1), 15, (255,0,0),cv2.FILLED)
        cv2.circle(img, (x2,y2), 15, (255,0,0),cv2.FILLED)
        
        # Drawing a line between the fingertips
        cv2.line(img, (x1,y1), (x2,y2), (0,255,0), 3)
        # Calculating the distance between the index finger and thumb
        length = hypot(x2-x1, y2-y1)

        # Converting finger range to volume range

        vol = np.interp(length, [15,220], [vol_min, vol_max])
        print(vol,length)

        # Setting the volume
        volume.SetMasterVolumeLevel(vol, None)

    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


-28.17283589845095 131.48764200486676
-27.774108730948527 132.74034804836094
-26.058503164818006 138.13037319865606
-26.62205788841156 136.35981812836215
0.0 270.44038160008574
0.0 283.09715646752795
0.0 243.46252278328177
0.0 243.46252278328177
0.0 247.39644298170498
0.0 245.52189311749777
0.0 246.34325645326686
0.0 247.34186867572583
0.0 246.34325645326686
0.0 248.16325271885037
0.0 244.13111231467406
0.0 231.00216449202375
-8.828534779206798 192.26284092356485
-18.16514402108954 162.92943257741985
-28.744393849522176 129.6919426949878
-30.186910600298596 125.15989773086265
-34.48234074099193 111.66467659918243
-37.908358857063064 100.90094152187085
-34.69390243902439 111.0
-39.466634953504155 96.00520819205592
-37.30829938435322 102.7861858422619
-39.21405311681809 96.79876032264049
-41.39042950469427 89.96110270555825
-43.12724603218389 84.50443775329198
-40.220227544411486 93.637599285757
-38.75709520825977 98.2344135219425
-20.05142536370635 157.00318468107582
-17.221370491212404

KeyboardInterrupt: 